In [1]:
%matplotlib inline

import math
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#### API reference
  - https://www.tensorflow.org/api_docs/python/tf/contrib/layers  

### Load MNIST data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Problem 1. 아래의 구조를 가지는 CNN 작성
  - Network (2 convolutional followed by 2 fully connected (linear) layer)
    - conv1: output_dim (32), filter_size (3,3), stride (2) without dropout
    - max_pool1: filter size (2,2), stride (2)
    - conv2: output_dim (64), filter_size (3,3) with dropout (keep_prob=0.8)
    - max_pool2: filter size (2,2), stride (2)
    - flat_feat: linear 연산을 위해 convolutional layer를 통과한 값을 (1, *)의 벡터로 변환, 결과적으로 (batch_size, *)의 매트릭스를 반환
    - fc1: linear layer (output_dim: 256) with dropout (keep_prob=0.8)
    - fc2: linear layer (output_dim: # of labels) without relu, dropout

In [3]:
def buildInference(images, num_labels):
    """ Build CNN network
    Args:
        images: 인풋 이미지
        num_lables: 레이블 수
    Returns:
        return CNN network 
    """
    # TODO: 아래 layer를 정의하세요.
    conv1     = tf.contrib.layers.conv2d(images, 32, [3, 3])
    max_pool1 = tf.contrib.layers.max_pool2d(conv1, [2, 2])
    conv2     = tf.contrib.layers.conv2d(max_pool1, 64, [3, 3],
                    normalizer_fn=tf.nn.dropout,
                    normalizer_params={'keep_prob': 0.8})
    max_pool2 = tf.contrib.layers.max_pool2d(conv2, [2, 2])

    flat_feat = tf.contrib.layers.flatten(max_pool2)

    # TODO: 아래 layer를 정의하세요.
    fc1 = tf.contrib.layers.fully_connected(flat_feat, 256)
    fc2 = tf.contrib.layers.fully_connected(fc1, num_labels, activation_fn=None)
    
    return tf.nn.softmax(fc2)

#### Problem 2. 각자의 CNN 모델 구성하기
  - filter size, layer 수 등을 바꿔서 구성해보세요.
  - 마지막 layer들은 위와 같이 (flat_feat -> fc1 -> fc2)가 되도록 해주세요.
  - 단 fc1의 output_dim은 바꾸셔도 됩니다.

#### 각자의 네트워크이므로 솔루션 제공 X

In [ ]:
def buildYourInference(images, num_labels):
    """ Build CNN network
    Args:
        images: 인풋 이미지
        num_lables: 레이블 수
    Returns:
        return CNN network 
    """
    # TODO: 아래 각자의 layer를 정의하세요.


    flat_feat = tf.contrib.layers.flatten(       )
    fc1 = 
    fc2 = 
    
    return tf.nn.softmax(fc2)

### Model Construction Functions

In [4]:
# Functions of model construction
def buildInputHolders():
    """ define placeholders for model inputs
    Returns:
        img_holder: images placeholder
        label_hodler: labels placeholder
    """
    img_holder = tf.placeholder(tf.float32, [None, 28, 28, 1])
    label_holder = tf.placeholder(tf.float32, [None, 10])
    
    return img_holder, label_holder

def buildLoss(preds, labels):
    """Calculates the loss from the predictions and the labels.
    Args:
        preds  : prediction tensor, [batch_size, NUM_CLASSES]
        labels : labels tensor, [batch_size]
    Returns:
        loss : loss tensor of type float
    """
    cross_entropy_loss = -tf.reduce_sum(labels * tf.log(preds), reduction_indices=[1])
    return tf.reduce_mean(cross_entropy_loss)

def buildAccuracy(pred, labels):
    """Calculates the accuracy from the predictions and the labels.
    Args:
        preds  : prediction tensor, [batch_size, NUM_CLASSES]
        labels : labels tensor, [batch_size]
    Returns:
        accuracy : accuracy tensor of type float
    """
    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

def buildModel(params):
    """ Build Model
    Args:
        params : dictionary data for parameters {
            'batch_size' : batch size
            'lr' : learning rate
            }
    Returns:
        imgs : input images of model
        labels : input labels of model
        train_step : one step operation for training
        loss : Loss tensor
        acc : accuracy
    """
    imgs, labels = buildInputHolders()
    preds = buildInference(imgs, 10)
    #preds = buildYourInference(imgs, 10)
    loss = buildLoss(preds, labels)
    acc = buildAccuracy(preds, labels)
    
    global_step = tf.Variable(.0, trainable=False)
    optimizer = tf.train.GradientDescentOptimizer(params['lr'])
    train_step = optimizer.minimize(loss, global_step=global_step)
    
    return imgs, labels, train_step, loss, acc

### Main Function

In [5]:
# define parameters
params = {}
params['batch_size'] = 100
params['lr'] = 0.1
params['total_batch'] = int(mnist.train.num_examples/params['batch_size'])

# build model
imgs, labels, train_step, loss, acc = buildModel(params)

In [6]:
batch_size = params['batch_size']
total_batch = params['total_batch']

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        total_cost = 0

        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 이미지 데이터를 CNN 모델을 위한 자료형태인 [height width channel] 의 형태로 재구성합니다.
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
            
            _, cost_val = sess.run([train_step, loss], \
                                   feed_dict={imgs: batch_xs, labels: batch_ys})
            total_cost += cost_val

        print "Epoch:", "%02d" % (epoch + 1), \
            "Avg. cost =", '{:.3f}'.format(total_cost / total_batch)

    # 테스트 데이터에 대한 정확도
    print "Test Accuracy: ", sess.run(acc, \
                    feed_dict={imgs: mnist.test.images.reshape(-1, 28, 28, 1), \
                               labels: mnist.test.labels})

Epoch: 01 Avg. cost = 0.305
Epoch: 02 Avg. cost = 0.087
Epoch: 03 Avg. cost = 0.061
Epoch: 04 Avg. cost = 0.047
Epoch: 05 Avg. cost = 0.039
Epoch: 06 Avg. cost = 0.032
Epoch: 07 Avg. cost = 0.028
Epoch: 08 Avg. cost = 0.023
Epoch: 09 Avg. cost = 0.020
Epoch: 10 Avg. cost = 0.018
Test Accuracy:  0.9892
